In [1]:
%load_ext autoreload
%autoreload 2

from simphony.libraries.ideal import Waveguide, Coupler
from simphony.circuit import Circuit
from simphony.simulation import QuantumSim, QuantumResult, CoherentState, SqueezedState, TwoModeSqueezed, compose_qstate
from simphony.simulation import ClassicalSim, Laser, Detector
from simphony.utils import polar
import numpy as jnp
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def coupler_factory():
    wg0 = Waveguide(length=1)
    wg1 = Waveguide(length=1)
    coupler = Coupler()
    ckt = Circuit()
    ckt.connect(coupler.o(2), wg0.o(0))
    ckt.connect(coupler.o(3), wg1.o(0))
    ckt.expose()
    return ckt
ckt = coupler_factory()
# print(pd.DataFrame(ckt.s_params(jnp.array([1.55]))[0]))
# input = TwoModeSqueezed(2, 10, 10, ckt.o(0), ckt.o(1))
coh1 = CoherentState(ckt, ckt.o(0), 1+2j)
coh2 = CoherentState(ckt, ckt.o(1), 3+4j)
input = compose_qstate(coh1, coh2)
sim = QuantumSim(ckt, wl=jnp.array([1.55]), input=input)
res = sim.run()
display(pd.DataFrame(res.input_means).T)
display(pd.DataFrame(res.means))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.0,1.494954,-3.075699,0.058719,0.176158,0.0,0.0,0.0,0.0,1.323374,-4.055913,0.117439,0.234877,0.0,0.0


In [3]:
def coupler_factory():
    wg0 = Waveguide(length=1)
    wg1 = Waveguide(length=1)
    coupler = Coupler()
    ckt = Circuit()
    ckt.connect(coupler.o(2), wg0.o(0))
    ckt.connect(coupler.o(3), wg1.o(0))
    ckt.expose()
    amp1, phase1 = polar(1+2j)
    amp2, phase2 = polar(3+4j)
    Laser(ckt, port=ckt.o(0), power=amp1**2, phase=phase1)
    Laser(ckt, port=ckt.o(1), power=amp2**2, phase=phase2)
    Detector(ckt, port=ckt.o(2))
    Detector(ckt, port=ckt.o(3))
    return ckt
ckt = coupler_factory()
wls = jnp.array([1.55])
sim = ClassicalSim(ckt=ckt, wl=wls)
res = sim.run()
display(pd.DataFrame(res.input_source))
display(pd.DataFrame(res.output))

,0,1,2,3
0,1.0+2.0j,3.0+4.0j,0.0+0.0j,0.0+0.0j


,0,1,2,3
0,0.0+0.0j,0.0+0.0j,1.494954+1.323374j,-3.075699-4.055913j


In [4]:
jnp.exp(1j*jnp.pi)

(-1+1.2246467991473532e-16j)